# Models in Pyro

这里Models注意指的是随机函数（stochastic funciton），其一般由两部分组成：

- 原始随机函数，即随机数生成器；
- 以随机数为输入的确定性的python代码。

在code层面上，随机函数一般被建构成函数、带有`__call__`方法的对象、方法或PyTorch `nn.Module`对象。

In [1]:
import torch
import pyro

pyro.set_rng_seed(101)

## 原始随机函数

就是随机分布生成器。

In [2]:
normal = torch.distributions.Normal(0, 1.0)  # 这个东西和scipy.stats.normal是一致的
x = normal.rsample()
print("sample", x)
print("log prob", normal.log_prob(x))

sample tensor(-1.3905)
log prob tensor(-1.8857)


当然，`pyro`中也有`pyro.distributions`，其就是对`torch.distributions`的一个wrapper。

In [3]:
normal2 = pyro.distributions.Normal(0, 1.0)
x = normal2.rsample()
print("sample", x)
print("log prob", normal2.log_prob(x))

sample tensor(-0.8152)
log prob tensor(-1.2512)


## 一个简单的model实例

这里可以理解模型为**现实生活中带有随机性数据的生成器**。

比如下面描述了温度的生成过程，其中影响它的主要因素是晴天还是阴天：

In [4]:
def wether():
    cloudy = torch.distributions.Bernoulli(0.3).sample()  # sample方法是正常的采样方法，无法进行反向传播
    cloudy = "cloudy" if cloudy.item() == 1.0 else "sunny"
    mean_temp = {"cloudy": 55.0, "sunny": 75.0}[cloudy]
    scale_temp = {"cloudy": 10.0, "sunny": 15.0}[cloudy]
    temp = torch.distributions.Normal(mean_temp, scale_temp).rsample()  # rsample是使用了重参数化技巧，可以进行反向传播，可以使用`has_rsample`方法来查看是否有此方法
    return cloudy, temp.item()

## pyro的model

上面的是使用`pytorch`的model，但如果我们希望使用`pyro`的工具，就需要：

- 使用`pyro.distributions`来替代`torch.distributions`；
- 使用`pyro.sample`函数替代原来的`.sample()`或`.rsample()`方法；

In [6]:
def wether():
    cloudy = pyro.sample("cloudy", pyro.distributions.Bernoulli(0.3))
    cloudy = "cloudy" if cloudy.item() == 1.0 else "sunny"
    mean_temp = {"cloudy": 55.0, "sunny": 75.0}[cloudy]
    scale_temp = {"cloudy": 10.0, "sunny": 15.0}[cloudy]
    temp = pyro.sample("temp", pyro.distributions.Normal(mean_temp, scale_temp))
    return cloudy, temp.item()

In [7]:
for _ in range(3):
    print(wether())

('cloudy', 64.5440444946289)
('sunny', 94.37557983398438)
('sunny', 72.5186767578125)


上述两个改变，除了保留了原始的功能外，实际上其还尽力了一个概率图模型，使得我们可以依照概率论的技术来进行后验推断等等。

## 递归调用model

Model本质上是一个函数，我们可以在另外的一个model中调用它来实现更大规模的概率图模型。

In [10]:
def ice_cream_sales():
    cloudy, temp = wether()
    expected_sales = 200. if cloudy == "sunny" and temp > 80.0 else 50.
    ice_cream = pyro.sample("ice_cream", pyro.distributions.Normal(expected_sales, 10.0))
    return ice_cream

In [11]:
for _ in range(3):
    print(ice_cream_sales())

tensor(220.6519)
tensor(210.4119)
tensor(44.5011)


我们甚至可以使用递归来建立model，只需要使用不同的名称即可。

In [12]:
def geometric(p, t=None):
    # 这里得到的是几何分布的一次采样，即单次为独立的伯努利分布，直到出现一次1之前出现0的次数
    if t is None:
        t = 0
    x = pyro.sample("x_{}".format(t), pyro.distributions.Bernoulli(p))
    if x.item() == 1:
        return 0
    else:
        return 1 + geometric(p, t + 1)

In [13]:
print(geometric(0.5))

1


或者定义model接受其他model随机的sample作为输入

In [15]:
def normal_product(loc, scale):
    z1 = pyro.sample("z1", pyro.distributions.Normal(loc, scale))
    z2 = pyro.sample("z2", pyro.distributions.Normal(loc, scale))
    y = z1 * z2
    return y

def make_normal_normal():
    mu_latent = pyro.sample("mu_latent", pyro.distributions.Normal(0, 1))
    fn = lambda scale: normal_product(mu_latent, scale)
    return fn

print(make_normal_normal()(1.0))

tensor(9.8305)


**以上说明，`pyro` Models可以利用几乎所有的python语法，类似`PyTorch`。**